<a href="https://colab.research.google.com/github/MichelyTamessawa/caixa-eletronico/blob/master/classificacao_laranjas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
siglas_laranjas = ["B", "C", "D", "P", "V"]
classes_laranjas = ["Boa", "Casca Grossa", "Dano Praga", "Podre", "Verde"]
path = "/content/drive/MyDrive/4 ano/tcc/classificação de laranjas/"

**Leitura do Dataset**

In [3]:
def leitura(tipo):
  boa = pd.DataFrame()
  casca_grossa = pd.DataFrame()
  dano_praga = pd.DataFrame()
  podre = pd.DataFrame()
  verde = pd.DataFrame()

  for index, classe in enumerate(classes_laranjas):
    for i in range(1, 11):
      path_resultado = path + tipo + classe + "/Fold " + str(i) + siglas_laranjas[index] + ".csv"
      linha = pd.read_csv(path_resultado)
      if classe == "Boa":
          boa = pd.concat([boa, linha],ignore_index=True)
      elif classe == "Casca Grossa":
          casca_grossa = pd.concat([casca_grossa, linha],ignore_index=True)
      elif classe == "Dano Praga":
          dano_praga = pd.concat([dano_praga, linha],ignore_index=True)
      elif classe == "Podre":
          podre = pd.concat([podre, linha],ignore_index=True)
      elif classe == "Verde":
          verde = pd.concat([verde, linha],ignore_index=True)
  
  return boa, casca_grossa, dano_praga, podre, verde

**Função que faz a divisão de inteiros por 4**

In [4]:
def divide_quatro_partes(num):
  resto = num % 4

  if resto == 0:
      return [num//4, num//4, num//4, num//4]
  elif resto == 3:
      return [num//4+1, num//4+1, num//4+1, num//4]
  elif resto == 2:
      return [num//4+1, num//4+1, num//4, num//4]
  elif resto == 1:
      return [num//4+1, num//4, num//4, num//4]

**Balanceamento das classes**

In [5]:
def boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = boa.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  c = casca_grossa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)

  classe_0 = boa
  classe_1 = pd.concat([c, d, p, v], axis=0, ignore_index=True)
  classe_1["class"].replace({2:1, 3:1, 4:1}, inplace=True)

  df = pd.concat([classe_0, classe_1], axis=0, ignore_index=True)
  X = df.drop('class', axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = casca_grossa.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  d = dano_praga.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)

  classe_0 = casca_grossa
  classe_0["class"] = 0

  classe_1 = pd.concat([b, d, p, v], axis=0, ignore_index=True)
  classe_1["class"].replace({0: 1, 2: 1, 3: 1, 4: 1}, inplace=True)
  
  df = pd.concat([classe_0, classe_1], axis=0, ignore_index=True)
  X = df.drop('class', axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
  
  return x_train, x_test, y_train, y_test


def dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = dano_praga.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  p = podre.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)

  classe_0 = dano_praga
  classe_0["class"] = 0
  
  classe_1 = pd.concat([c, b, p, v], axis=0, ignore_index=True)
  classe_1["class"].replace({0: 1, 3: 1, 4: 1}, inplace=True)
    
  df = pd.concat([classe_0, classe_1], axis=0, ignore_index=True)
  X = df.drop('class', axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = podre.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
  v = verde.sample(n=divisao_classes[3],random_state=1)

  classe_0 = podre
  classe_0["class"] = 0
  
  classe_1 = pd.concat([c, b, d, v], axis=0, ignore_index=True)
  classe_1["class"].replace({0: 1, 2: 1, 4: 1}, inplace=True)
    
  df = pd.concat([classe_0, classe_1], axis=0, ignore_index=True)

  X = df.drop('class', axis=1)
  y = df["class"]
      
  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test


def verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde):
  n_classe  = verde.shape[0]
  divisao_classes = divide_quatro_partes(n_classe)

  b = boa.sample(n=divisao_classes[0],random_state=1)
  c = casca_grossa.sample(n=divisao_classes[1],random_state=1)
  d = dano_praga.sample(n=divisao_classes[2],random_state=1)
  p = podre.sample(n=divisao_classes[3],random_state=1)

  classe_0 = verde
  classe_0["class"] = 0
  
  classe_1 = pd.concat([c, b, p, d], axis=0, ignore_index=True)
  classe_1["class"].replace({0: 1, 2: 1, 3: 1}, inplace=True)  

  df = pd.concat([classe_0, classe_1], axis=0, ignore_index=True)

  X = df.drop('class', axis=1)
  y = df["class"]

  x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

  return x_train, x_test, y_train, y_test

**Classificador K-NN**

In [6]:
def knn_gridsearchcv(x_train, x_test, y_train, y_test):
  knn = KNeighborsClassifier()

  parameters = {
      'n_neighbors': range(1,30),
      'leaf_size': (20,40,1),
      'p': (1,2),
      'weights': ('uniform', 'distance'),
      'metric': ('minkowski', 'chebyshev') }

  # selecionando os melhores parâmetros para o modelo
  grid = GridSearchCV(estimator=knn, param_grid=parameters, cv=10, refit='True', scoring='accuracy')
  grid_search = grid.fit(x_train, y_train)
  print("Parâmetros: {}".format(grid_search.best_params_))

  # acurácia do modelo com os dados de treino
  accuracy = grid_search.best_score_ *100    
  print("Acurácia para os dados de treinamento: {:.2f}%".format(accuracy))
  
  # acurácia com os dados de teste
  y_test_result = grid.predict(x_test)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acurácia dos testes: {:.2f}%".format(test_accuracy))

  print("Matriz de Confusão:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

**Classificador Random Forest**

In [7]:
def rf_gridsearchcv(x_train, x_test, y_train, y_test):
  rf = RandomForestClassifier()

  param_grid = {
      'bootstrap': [True],
      'max_depth': [80, 90, 100, 110],
      'max_features': [2, 3],
      'min_samples_leaf': [3, 4, 5],
      'min_samples_split': [8, 10, 12],
      'n_estimators': [100, 200, 300, 1000]
  }

  # Treinamento com gridsearch
  grid = GridSearchCV(rf, param_grid, cv=10, refit='True', scoring='accuracy')
  grid_search = grid.fit(x_train, y_train)
  print("Parâmetros: {}".format(grid_search.best_params_))

  # Acurácia do modelo com os dados de treino
  accuracy = grid_search.best_score_ *100    
  print("Acurácia no treinamento: {:.2f}%".format(accuracy))

  # Teste
  y_test_result = grid.predict(x_test)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acurácia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusão
  print("Matriz de Confusão:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))


**Classificador SVM**

In [8]:
def svm_gridsearchcv(x_train, x_test, y_train, y_test):
  clf = SVC()

  param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}
              
  # Treinamento com gridsearch
  grid = GridSearchCV(clf, param_grid, cv=10, refit='True', scoring='accuracy')
  grid_search = grid.fit(x_train, y_train)
  print("Parâmetros: {}".format(grid_search.best_params_))

  # Acurácia do modelo com os dados de treino
  accuracy = grid_search.best_score_ *100    
  print("Acurácia no treinamento: {:.2f}%".format(accuracy))

  # Teste
  y_test_result = grid.predict(x_test)
  test_accuracy = accuracy_score(y_test, y_test_result)*100
  print("Acurácia dos testes: {:.2f}%".format(test_accuracy))

  # Matriz de Confusão
  print("Matriz de Confusão:")
  print (pd.crosstab(y_test, y_test_result, rownames=['Real'], colnames=['Predito'], margins=True))

**K-NN: Tons de Cinza**

In [ ]:
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_cinza/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 66.33%
Acurácia dos testes: 67.29%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        339  132  471
1        165  272  437
All      504  404  908


Casca Grossa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 20, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 67.62%
Acurácia dos testes: 68.50%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        361  110  471
1        176  261  437
All      537  371  908


Dano Praga vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 27, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 63.43%
Acurácia dos testes: 64.19%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        322  142  464
1        176  248  424
All      498  390  888


Pod

**K-NN: RGB**

In [9]:
boa, casca_grossa, dano_praga, podre, verde = leitura("greycoprops_colorida/")
print("Boa vs Todos")
x_train, x_test, y_train, y_test = boa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Casca Grossa vs Todos")
x_train, x_test, y_train, y_test = casca_grossa_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Dano Praga vs Todos")
x_train, x_test, y_train, y_test = dano_praga_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Podre vs Todos")
x_train, x_test, y_train, y_test = podre_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)
print("\n")
print("Verde vs Todos")
x_train, x_test, y_train, y_test = verde_vs_todos(boa, casca_grossa, dano_praga, podre, verde)
knn_gridsearchcv(x_train, x_test, y_train, y_test)

Boa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 11, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 78.22%
Acurácia dos testes: 80.18%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        413   58  471
1        122  315  437
All      535  373  908


Casca Grossa vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 86.12%
Acurácia dos testes: 86.78%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        428   43  471
1         77  360  437
All      505  403  908


Dano Praga vs Todos
Parâmetros: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Acurácia para os dados de treinamento: 85.61%
Acurácia dos testes: 86.60%
Matriz de Confusão:
Predito    0    1  All
Real                  
0        429   35  464
1         84  340  424
All      513  375  888


Podre